In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LeakyReLU, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# --- 1. Data Generation ---
def generate_real_samples(n_samples=1000):
    """
    Generates a dataset based on a quadratic function y = x^2 with some noise.
    """
    # Generate random x values
    x1 = np.random.rand(n_samples) * 4 - 2 # Range from -2 to 2
    # Generate y values based on the quadratic function
    x2 = x1 * x1 + np.random.normal(0, 0.1, n_samples) # Add some Gaussian noise
    # Reshape and stack
    x1 = x1.reshape(n_samples, 1)
    x2 = x2.reshape(n_samples, 1)
    X = np.hstack((x1, x2))
    # Generate class labels
    y = np.ones((n_samples, 1))
    return X, y

# --- 2. Model Implementation ---
def build_generator(latent_dim, n_outputs=2):
    """
    Builds the Generator network.
    """
    in_lat = Input(shape=(latent_dim,))
    # 2 hidden layers
    hid1 = Dense(15, activation=LeakyReLU(alpha=0.2))(in_lat)
    hid2 = Dense(15, activation=LeakyReLU(alpha=0.2))(hid1)
    # Output layer
    out_layer = Dense(n_outputs, activation='linear')(hid2)
    model = Model(in_lat, out_layer)
    return model

def build_discriminator(n_inputs=2):
    """
    Builds the Discriminator network.
    """
    in_samp = Input(shape=(n_inputs,))
    # 3 hidden layers
    hid1 = Dense(25, activation=LeakyReLU(alpha=0.2))(in_samp)
    hid2 = Dense(25, activation=LeakyReLU(alpha=0.2))(hid1)
    # Third hidden layer with 2 nodes for visualization
    hid3 = Dense(2, activation=LeakyReLU(alpha=0.2))(hid2)
    # Output layer
    out_layer = Dense(1, activation='sigmoid')(hid3)
    model = Model(in_samp, out_layer)
    # Compile model
    opt = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

def build_gan(generator, discriminator):
    """
    Defines the combined GAN model for training the generator.
    """
    # Make weights in the discriminator not trainable
    discriminator.trainable = False
    # Connect them
    model = Model(generator.input, discriminator(generator.output))
    # Compile model
    opt = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# --- Utility functions for training and plotting ---
def generate_latent_points(latent_dim, n_samples):
    """Generate points in latent space as input for the generator."""
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

def generate_fake_samples(generator, latent_dim, n_samples):
    """Use the generator to generate n fake examples, with class labels."""
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    return X, y

def summarize_performance(iteration, generator, discriminator, latent_dim, n_samples=100):
    """Evaluate the discriminator, plot generated samples, and save the plot."""
    # Prepare real samples
    X_real, y_real = generate_real_samples(n_samples)
    # Evaluate discriminator on real examples
    _, acc_real = discriminator.evaluate(X_real, y_real, verbose=0)
    # Prepare fake examples
    x_fake, y_fake = generate_fake_samples(generator, latent_dim, n_samples)
    # Evaluate discriminator on fake examples
    _, acc_fake = discriminator.evaluate(x_fake, y_fake, verbose=0)
    # Summarize discriminator performance
    print(f'>Iteration {iteration}, Real Acc: {acc_real*100:.2f}%, Fake Acc: {acc_fake*100:.2f}%')
    # Scatter plot of real and fake data
    plt.figure(figsize=(8, 8))
    plt.scatter(X_real[:, 0], X_real[:, 1], color='blue', label='Real Data', alpha=0.7)
    plt.scatter(x_fake[:, 0], x_fake[:, 1], color='red', label='Generated Data', alpha=0.7)
    plt.title(f'Real vs. Generated Samples (Iteration {iteration})')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.legend()
    plt.grid(True)
    plt.xlim(-2.5, 2.5)
    plt.ylim(-0.5, 4.5)
    # Save the plot to a file instead of showing it
    plt.savefig(f'gan_generated_plot_iter_{iteration}.png')
    plt.close() # Close the figure to free up memory

# --- 3. Training Loop ---
def train(g_model, d_model, gan_model, latent_dim, n_iter=10001, n_batch=128):
    """
    Train the generator and discriminator.
    """
    half_batch = int(n_batch / 2)
    d_losses, g_losses = [], []
    # Manually enumerate iterations
    for i in range(n_iter):
        # --- Train Discriminator ---
        # Get randomly selected 'real' samples
        X_real, y_real = generate_real_samples(half_batch)
        # Generate 'fake' examples
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # Update discriminator model weights
        d_loss_real, _ = d_model.train_on_batch(X_real, y_real)
        d_loss_fake, _ = d_model.train_on_batch(X_fake, y_fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # --- Train Generator ---
        # Prepare points in latent space as input for the generator
        X_gan = generate_latent_points(latent_dim, n_batch)
        # Create inverted labels for the fake samples
        y_gan = np.ones((n_batch, 1))
        # Update the generator via the discriminator's error
        g_loss = gan_model.train_on_batch(X_gan, y_gan)

        # Store losses
        d_losses.append(d_loss)
        g_losses.append(g_loss)

        # Evaluate the model performance, sometimes
        if (i % 1000) == 0:
            summarize_performance(i, g_model, d_model, latent_dim)

    # --- 4. Plotting Losses ---
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label='Discriminator Loss')
    plt.plot(g_losses, label='Generator Loss')
    plt.title('Training Losses')
    plt.xlabel('Iteration')
    plt.ylabel('Loss (Binary Cross-Entropy)')
    plt.legend()
    plt.grid(True)
    plt.show()


if __name__ == '__main__':
    # Size of the latent space
    latent_dim = 5
    # Create the discriminator
    discriminator = build_discriminator()
    # Create the generator
    generator = build_generator(latent_dim)
    # Create the gan
    gan_model = build_gan(generator, discriminator)
    # Train model
    print("--- Starting GAN Training ---")
    train(generator, discriminator, gan_model, latent_dim)
    print("--- Training Complete ---")



--- Starting GAN Training ---
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
>Iteration 0, Real Acc: 56.00%, Fake Acc: 33.00%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━

KeyboardInterrupt: 